In [3]:
#####################################
# IMPORTING LIBRERIE
#####################################
import os
import gc
import random
import torch
import time
import json
import pandas as pd
import numpy as np
from IPython.display import Markdown
from datasets import load_dataset, Dataset, concatenate_datasets
from tqdm import tqdm

In [4]:
def init_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

init_seed(33)

In [5]:
DATASET = "yuan-yang/MALLS-v0"
malls_dataset = load_dataset(DATASET, split="train").to_pandas()
nl_to_fol_df = malls_dataset.sample(n=10000, random_state=42).reset_index(drop=True)
fol_to_nl_df = malls_dataset.sample(n=10000, random_state=123).reset_index(drop=True)

README.md: 0.00B [00:00, ?B/s]

MALLS-v0.1-train.json: 0.00B [00:00, ?B/s]

MALLS-v0.1-test.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/27284 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
combined_samples = pd.concat([nl_to_fol_df, fol_to_nl_df]).drop_duplicates()

# Rimuovere le righe presenti nei sample dal dataset originale
remaining_dataset = malls_dataset.merge(combined_samples, how='outer', indicator=True)
remaining_dataset = remaining_dataset[remaining_dataset['_merge'] == 'left_only'].drop(columns=['_merge'])

# Campionare 3.000 istanze
final_sample = remaining_dataset.sample(n=1000, random_state=7).reset_index(drop=True)


In [7]:
dataset = Dataset.from_pandas(final_sample)
dataset

Dataset({
    features: ['FOL', 'NL'],
    num_rows: 1000
})

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

model_id = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
token = ""
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

print("Caricamento del modello e del tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    token=token,
    quantization_config=quantization_config
)

2025-07-15 13:35:54.038966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752586554.229243      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752586554.284410      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Caricamento del modello e del tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [9]:
adapter_id = "francescoocurcio/new_llama3.2-3B-log-ftn-malls-3epoch_10k-sysprompt_no"
model.load_adapter(adapter_id, adapter_name="logic")

adapter_config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

In [10]:
def build_prompt(text):
    messages = [
        {
            "role": "user",
            "content": (
                f"{text.strip()}\n\n"
                "Provide only the First-Order Logic formula. Do not include explanations or introductions.\n"
                "For example:\n"
                "Input: All humans are mortal.\n"
                "Output: ∀x (Human(x) → Mortal(x))\n"
            )
        }
    ]

    chat_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
        
    return chat_input

In [ ]:
def generator(prompt, model, tokenizer, max_new_tokens=1024):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    input_text = tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)
    generated_text = full_output[len(input_text):].strip()

    return generated_text

In [12]:
fol_results = []

for i, (_, row) in enumerate(tqdm(final_sample.iterrows(), total=len(final_sample))):
    nl_text = row["NL"]
    fol_true = row['FOL']
    prompt = build_prompt(nl_text)

    output = generator(prompt, model, tokenizer, max_new_tokens=512)

    fol_response = output.split("Output:")[-1].strip()

    if i < 10:
        print(f"NL:{nl_text}, FOL:{fol_true}")
        print(f"Input (prompt):\n{prompt}\n")
        print(f"Output (raw):\n{output}\n")
        print(f"Response post-processata:\n{fol_response}\n")
        print("="*40)

    fol_results.append(fol_response)

final_sample["output"] = fol_results
final_sample.to_csv("nl_to_fol_with_gemma.csv", index=False)

  0%|          | 1/1000 [00:07<2:10:16,  7.82s/it]

NL:Mercury is closer to the sun than Venus, and Venus is closer to the sun than Earth., FOL:∀x∀y∀z (Mercury(x) ∧ Venus(y) ∧ Earth(z) → (CloserToSun(x, y) ∧ CloserToSun(y, z)))
Input (prompt):
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 15 Jul 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Mercury is closer to the sun than Venus, and Venus is closer to the sun than Earth.

Provide only the First-Order Logic formula. Do not include explanations or introductions.
For example:
Input: All humans are mortal.
Output: ∀x (Human(x) → Mortal(x))<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Output (raw):
∀x ∀y ∀z (Mercury(x) ∧ Venus(y) ∧ Earth(z) → (CloserToSun(x, sun) ∧ CloserToSun(y, sun) ∧ CloserToSun(z, sun)))

Response post-processata:
∀x ∀y ∀z (Mercury(x) ∧ Venus(y) ∧ Earth(z) → (CloserToSun(x, sun) ∧ CloserToSun(y, sun) ∧ CloserToSun(z, sun)))



  0%|          | 2/1000 [00:12<1:38:30,  5.92s/it]

NL:A celestial event that occurs when the Moon passes between the Earth and the Sun, blocking the Sun's light, is a solar eclipse., FOL:∀x (CelestialEvent(x) ∧ MoonPassesBetween(x) ∧ EarthAndSun(x) ∧ BlocksSunlight(x) → SolarEclipse(x))
Input (prompt):
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 15 Jul 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

A celestial event that occurs when the Moon passes between the Earth and the Sun, blocking the Sun's light, is a solar eclipse.

Provide only the First-Order Logic formula. Do not include explanations or introductions.
For example:
Input: All humans are mortal.
Output: ∀x (Human(x) → Mortal(x))<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Output (raw):
∀x (CelestialEvent(x) ∧ MoonPassesBetweenEarthAndSun(x) ∧ BlocksSunlight(x) → SolarEclipse(x))

Response post-processata:
∀x (CelestialEvent(x) ∧ MoonPassesBetweenEarthAndSun(x) ∧ BlocksSunlight(x) → Sol

  0%|          | 3/1000 [00:16<1:23:01,  5.00s/it]

...

  1%|          | 9/1000 [00:41<1:10:36,  4.27s/it]

NL:A plant that is characterized by its green, leaf-like structures and its ability to produce food through photosynthesis is an algae., FOL:∀x (Plant(x) ∧ GreenLeafLikeStructures(x) ∧ ProducesFoodThroughPhotosynthesis(x) → Algae(x))
Input (prompt):
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 15 Jul 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

A plant that is characterized by its green, leaf-like structures and its ability to produce food through photosynthesis is an algae.

Provide only the First-Order Logic formula. Do not include explanations or introductions.
For example:
Input: All humans are mortal.
Output: ∀x (Human(x) → Mortal(x))<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Output (raw):
∀x (Plant(x) ∧ GreenLeafLikeStructures(x) ∧ Photosynthesis(x) → Algae(x))

Response post-processata:
∀x (Plant(x) ∧ GreenLeafLikeStructures(x) ∧ Photosynthesis(x) → Algae(x))



  1%|          | 10/1000 [00:45<1:06:47,  4.05s/it]

NL:A fruit can be sweet or sour, but it cannot be completely tasteless., FOL:∀x (Fruit(x) → ((Sweet(x) ∨ Sour(x)) ∧ ¬Tasteless(x)))
Input (prompt):
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 15 Jul 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

A fruit can be sweet or sour, but it cannot be completely tasteless.

Provide only the First-Order Logic formula. Do not include explanations or introductions.
For example:
Input: All humans are mortal.
Output: ∀x (Human(x) → Mortal(x))<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Output (raw):
∀x (Fruit(x) → (Sweet(x) ∨ Sour(x))) ∧ ¬Tasteless(x)

Response post-processata:
∀x (Fruit(x) → (Sweet(x) ∨ Sour(x))) ∧ ¬Tasteless(x)



100%|██████████| 1000/1000 [1:12:24<00:00,  4.34s/it]


In [13]:
dataset_to_push = Dataset.from_pandas(final_sample)

In [14]:
dataset_to_push.push_to_hub("stefra/llama")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/169k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/stefra/llama/commit/005c052844c774a4481473219881f66399bfc403', commit_message='Upload dataset', commit_description='', oid='005c052844c774a4481473219881f66399bfc403', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/stefra/llama', endpoint='https://huggingface.co', repo_type='dataset', repo_id='stefra/llama'), pr_revision=None, pr_num=None)